This notebook will figure out how to correctly format an API call to the visual crossing weather app. Documentation is here: 
>https://www.visualcrossing.com/resources/documentation/weather-api/timeline-weather-api/

The goal is to have a function that can be given the lat/long and date/time of an event and return the weather conditions.

In [2]:
# get API tokens
with open('.env.development') as f:
    data = f.read()

for line in data.split('\n'):
    head,sep,tail = line.partition(' = ')
    
    if head == 'crashAPIkey':
        crashAPIkey = tail
    elif head == 'weatherAPIkey':
        weatherAPIkey = tail

#print(f'CrashAPIToken {crashAPIkey},WeatherAPIToken {weatherAPIkey} ')

In [4]:
lat = '39.711714'
lon = '-90.191958'
date = '2020-12-15'
time = '14:00:00'

In [5]:
ApiQuery = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'+\
            lat+'%2C'+lon+'/'+date+'T'+time+'?unitGroup=us&key='+weatherAPIkey+'&include=current&contentType=json'

print(ApiQuery)

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/39.711714%2C-90.191958/2020-12-15T14:00:00?unitGroup=us&key=EJFB8QG6H8BUGKYGSSY4PF4BQ&include=current&contentType=json


In [8]:
import urllib
try: 
    with urllib.request.urlopen(ApiQuery) as response:
        html = response.read()
except urllib.error.HTTPError  as e:
    ErrorInfo= e.read().decode() 
    print('Error code: ', e.code, ErrorInfo)
    sys.exit()
except  urllib.error.URLError as e:
    ErrorInfo= e.read().decode() 
    print('Error code: ', e.code,ErrorInfo)
    sys.exit()

In [ ]:
import json
weather = json.loads(html.decode('utf-8'))

condistions at lat/long date/time:
* 'temp': 30.9,
* 'feelslike': 22.3,
* 'humidity': 77.1,
* 'dew': 24.6,
* 'precip': 0.0,
* 'precipprob': 0.0,
* 'snow': 0.0,
* 'snowdepth': 0.0,
* 'preciptype': None,
* 'windgust': None,
* 'windspeed': 10.2,
* 'winddir': 60.0,
* 'pressure': 1023.9,
* 'visibility': 9.9,
* 'cloudcover': 99.7,
* 'solarradiation': 99.0,
* 'solarenergy': 0.4,
* 'uvindex': 1.0,
* 'conditions': 'Overcast',
* 'icon': 'cloudy',
* 'stations': ['74466653944','KSPI','KPPQ','74466353950', '72439093822','KIJX'],
* 'source': 'obs',
* 'sunrise': '07:15:05',
* 'sunriseEpoch': 1608038105,
* 'sunset': '16:37:23',
* 'sunsetEpoch': 1608071843,
* 'moonphase': 0.01}

In [23]:
for item in ['temp','windspeed','winddir','preciptype','conditions']:
    val = weather['currentConditions'][item]
    print(f'{item} reported to be {val}')

temp reported to be 30.9
windspeed reported to be 10.2
winddir reported to be 60.0
preciptype reported to be None
conditions reported to be Overcast


How to add these to a DF?
1) Define a function that takes the relevant values from the DF and returns whatever info is desired from HTML lookup

2) Add it to the DF

In [3]:
def weatherLookup(lat,lon,dateTime,APIkey=weatherAPIkey):
    import urllib
    import json
    # the data in the DF is float and dateTime, which needs to be convered to strings to be added to the API query
    latS, lonS  = str(lat), str(lon)
    dateTimeS = dateTime.strftime('%Y-%m-%dT%H:%M:%S') #YYYY-MM-DDTHH:MM:SS
    ApiQuery = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'+\
            latS+'%2C'+lonS+'/'+dateTimeS+'?unitGroup=us&key='+APIkey+'&include=current&contentType=json'
    try: 
        with urllib.request.urlopen(ApiQuery) as response:
            html = response.read()
    except urllib.error.HTTPError  as e:
        ErrorInfo= e.read().decode() 
        print('Error code: ', e.code, ErrorInfo)
        sys.exit()
    except  urllib.error.URLError as e:
        ErrorInfo= e.read().decode() 
        print('Error code: ', e.code,ErrorInfo)
        sys.exit()

    weather = json.loads(html.decode('utf-8'))
    return weather['currentConditions']['temp'], weather['currentConditions']['conditions'], weather['currentConditions']['preciptype']

In [ ]:
# add three columns to the DF with the values returned from the weatherLookup function
test[['wTemp','wCond','wPrecip']] = \
    [weatherLookup(*a) for a in tuple(zip(test['latitude'],test['longitude'],test['crash_date']))]